In [2]:
%autoreload 2
import mxnet as mx
import pickle as pkl
import voc_utils.voc_utils as voc

In [ ]:
# use params from googlenet, but train for multilabel
# using pascal voc
df = voc.load_data_multilabel('train')
# df['filename'] = voc.img_dir + df['filename']
df['filename'] += '.jpg'
# true_outputs = df.loc[:, 'aeroplane':'tvmonitor'].values
print df.columns
print voc.img_dir
print len(voc.list_image_sets())

In [7]:
# use params from googlenet, but train for multilabel
# using pascal voc
df = voc.load_data_multilabel('val')
# df['filename'] = voc.img_dir + df['filename']
df['filename'] += '.jpg'
# true_outputs = df.loc[:, 'aeroplane':'tvmonitor'].values
print df.columns
print voc.img_dir
print len(voc.list_image_sets())

Index([u'filename', u'aeroplane', u'bicycle', u'bird', u'boat', u'bottle',
       u'bus', u'car', u'cat', u'chair', u'cow', u'diningtable', u'dog',
       u'horse', u'motorbike', u'person', u'pottedplant', u'sheep', u'sofa',
       u'train', u'tvmonitor'],
      dtype='object')
/Users/mprat/personal/VOCdevkit/VOCdevkit/VOC2012/JPEGImages/
20


In [ ]:
# make the data iter that can be used for training
df.to_csv('voc_train.lst', sep='\t', header=False, columns=['aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
       'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog',
       'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa',
       'train', 'tvmonitor', 'filename'])

In [8]:
# make the data iter that can be used for training
df.to_csv('voc_val.lst', sep='\t', header=False, columns=['aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
       'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog',
       'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa',
       'train', 'tvmonitor', 'filename'])

In [ ]:
!./../mxnet/bin/im2rec voc_train.lst /Users/mprat/personal/VOCdevkit/VOCdevkit/VOC2012/JPEGImages/ voc_train.rec resize=200, label_width=20

In [9]:
!./../mxnet/bin/im2rec voc_val.lst /Users/mprat/personal/VOCdevkit/VOCdevkit/VOC2012/JPEGImages/ voc_val.rec resize=200, label_width=20

[14:17:46] tools/im2rec.cc:54: New Image Size: Short Edge 200
[14:17:46] tools/im2rec.cc:76: Write to output: voc_val.rec
[14:17:46] tools/im2rec.cc:78: Output: voc_val.rec
[14:17:46] tools/im2rec.cc:86: JPEG encoding quality: 80
[14:17:55] tools/im2rec.cc:157: 1000 images processed, 9.55097 sec elapsed
[14:18:05] tools/im2rec.cc:157: 2000 images processed, 19.4967 sec elapsed
[14:18:16] tools/im2rec.cc:157: 3000 images processed, 29.5878 sec elapsed
[14:18:26] tools/im2rec.cc:157: 4000 images processed, 39.6302 sec elapsed
[14:18:36] tools/im2rec.cc:157: 5000 images processed, 49.6172 sec elapsed
[14:18:44] tools/im2rec.cc:160: Total: 5823 images processed, 58.0862 sec elapsed


In [ ]:
imgs = voc.load_imgs(df['filename'])

In [3]:
dataiter = mx.io.ImageRecordIter(
    path_imgrec='voc_train.rec',
    data_shape=(3, 200, 200),
    path_imglist='voc_train.lst',
    label_width=len(voc.list_image_sets()),
    shuffle=True,
    batch_size=20)

In [10]:
datavaliter = mx.io.ImageRecordIter(
    path_imgrec='voc_val.rec',
    data_shape=(3, 200, 200),
    path_imglist='voc_val.lst',
    label_width=len(voc.list_image_sets()),
    shuffle=True,
    batch_size=20)

In [5]:
# create multilabel net and train from scratch
input_data = mx.symbol.Variable('data')
conv1 = mx.symbol.Convolution(data=input_data, name='conv1', kernel=(20, 20), stride=(20, 20), pad=(0, 0), num_filter=100)
relu1 = mx.symbol.Activation(data=conv1, name='relu1', act_type='relu')
top_pool = mx.symbol.Pooling(data=relu1, pool_type='max', kernel=(5, 1), stride=(2, 2), pad=(0, 0))
out = mx.symbol.SoftmaxOutput(data=top_pool, name='softmax', multi_output=True)
net = mx.model.FeedForward(symbol=out, num_epoch=2, learning_rate=0.01)
net.fit(X=dataiter, eval_data=datavaliter)